In [1]:
import os
import pprint
import random
import numpy as np
import torch
import torch.nn.parallel
import torch.optim
import itertools
import argparse
import json
import tqdm

from config.config import config, update_config
from utils import exp_utils
from evaluation import eval_utils

import matplotlib.pyplot as plt
from scipy.signal import find_peaks, medfilt

/vision/hwjiang/anaconda3/envs/vq2d/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mode = 'val'
annotation_path = os.path.join('/vision/hwjiang/episodic-memory/VQ2D/data', 'vq_{}.json'.format(mode))
with open(annotation_path) as fp:
    annotations = json.load(fp)
clipwise_annotations_list = eval_utils.convert_annotations_to_clipwise_list(annotations)

In [3]:
cache_path ='/vision/hwjiang/vq2d/output/ego4d_vq2d/vq2d_all_transformer2_anchor_dinov2_inference_egotracks/default-arch_aug-no_anchor_focal-w10_2heads_dropout0.2_positive0.3_egotrack_fix-assign_res16_inference/inference_cache_val'

In [4]:
def draw_scores(pred, gt, annot_key, name):
    '''
    both in shape [N]
    '''
    assert name in ['raw', 'smooth', 'smooth_thresh']
    save_path = './visualization/plot_scores'
    os.makedirs(save_path, exist_ok=True)

    N = pred.shape[0]
    x = np.arange(N)

    pred_sm = medfilt(pred, kernel_size=5)

    if name == 'raw':
        plt.plot(x, pred, marker=None, color='b', label='pred')
    elif name == 'smooth':
        plt.plot(x, pred_sm, marker=None, color='g', label='pred_sm')
    else:
        pred_sm[pred_sm < 0.5] = 0
        plt.plot(x, pred_sm, marker=None, color='g', label='pred_sm_thre')
    plt.plot(x, gt, marker=None, color='r', label='gt')
    plt.xlabel('number of frames')
    plt.ylabel('occurance score')
    plt.ylim((0.0, 1.05))
    plt.legend(loc='best')

    plt.savefig(os.path.join(save_path, annot_key + f'_{name}.jpg'))
    plt.close()

In [5]:
i = 20
for _, annots in clipwise_annotations_list.items():
    clip_uid = annots[0]["clip_uid"]
    clip_dir = '/vision/srama/Research/Ego4D/episodic-memory/VQ2D/data/clips_fullres'
    clip_path = os.path.join(clip_dir, clip_uid  + '.mp4')
    keys = [
            (annot["metadata"]["annotation_uid"], annot["metadata"]["query_set"])
            for annot in annots
        ]
    for key, annot in zip(keys, annots):

        annotation_uid = annot["metadata"]["annotation_uid"]
        query_set = annot["metadata"]["query_set"]
        annot_key = f"{annotation_uid}_{query_set}"
        query_frame = annot["query_frame"]
        visual_crop = annot["visual_crop"]
        
        save_path = os.path.join(cache_path, f'{annot_key}.pt')
        cache = torch.load(save_path)
        ret_bboxes, ret_scores = cache['ret_bboxes'], torch.sigmoid(cache['ret_scores'])
        print(ret_scores.shape, query_frame)

        gt_scores = torch.zeros_like(ret_scores)
        gt_idx = []
        for it in annot['response_track']:
            gt_idx.append(it['frame_number'])
        gt_scores[gt_idx] = 1.0

        draw_scores(ret_scores.numpy(), gt_scores.numpy(), annot_key, 'raw')
        #draw_scores(ret_scores.numpy(), gt_scores.numpy(), annot_key, 'smooth')
        #draw_scores(ret_scores.numpy(), gt_scores.numpy(), annot_key, 'smooth_thresh')
        i -= 1
    if i == 0:
        break

torch.Size([1433]) 1433
torch.Size([80]) 80
torch.Size([1787]) 1787
torch.Size([298]) 298
torch.Size([1044]) 1044
torch.Size([1498]) 1498
torch.Size([1499]) 1499
torch.Size([363]) 363
torch.Size([489]) 489
torch.Size([1088]) 1088
torch.Size([812]) 812
torch.Size([1402]) 1402
torch.Size([1952]) 1952
torch.Size([2054]) 2054
torch.Size([2339]) 2339
torch.Size([827]) 827
torch.Size([674]) 674
torch.Size([963]) 963
torch.Size([125]) 125
torch.Size([501]) 501
torch.Size([1306]) 1306
torch.Size([158]) 158
torch.Size([137]) 137
torch.Size([775]) 775
torch.Size([614]) 614
torch.Size([1650]) 1650
torch.Size([1652]) 1652
torch.Size([169]) 169
torch.Size([1530]) 1530
torch.Size([2287]) 2287
torch.Size([133]) 133
torch.Size([1032]) 1032
torch.Size([1817]) 1817
torch.Size([243]) 243
torch.Size([690]) 690
torch.Size([1251]) 1251
torch.Size([278]) 278
torch.Size([1499]) 1499
torch.Size([1499]) 1499
torch.Size([1389]) 1389
torch.Size([1805]) 1805
torch.Size([1618]) 1618
torch.Size([817]) 817
torch.Size

IndexError: index 1800 is out of bounds for dimension 0 with size 1800

In [22]:
annot['response_track'][0]['frame_number']

188